# 探索性数据分析 (EDA) 与特征工程

目标：理解数据分布、特征关系，确定特征工程方案

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 加载数据
df = pd.read_csv('admissions.csv')
print(f"数据集形状: {df.shape}")
print("\n数据预览:")
df.head()

In [ ]:
# 数据基本信息
print("数据基本信息:")
print(df.info())

print("\n缺失值统计:")
print(df.isnull().sum())

print("\n数值型字段统计:")
print(df.describe())

In [ ]:
# 目标变量分析
plt.figure(figsize=(15, 10))

# 热度指数分布
plt.subplot(2, 3, 1)
sns.histplot(df['hotness_index'], bins=30, kde=True)
plt.title('热度指数分布')
plt.xlabel('热度指数')

# 热度指数箱线图
plt.subplot(2, 3, 2)
sns.boxplot(y=df['hotness_index'])
plt.title('热度指数箱线图')

# 对数变换后的热度指数
plt.subplot(2, 3, 3)
log_hotness = np.log1p(df['hotness_index'])
sns.histplot(log_hotness, bins=30, kde=True)
plt.title('对数变换后的热度指数')
plt.xlabel('log(hotness_index + 1)')

# 热度指数与年份的关系
plt.subplot(2, 3, 4)
sns.boxplot(data=df, x='year', y='hotness_index')
plt.title('不同年份的热度指数')
plt.xticks(rotation=45)

# 热度指数与学校层次的关系
plt.subplot(2, 3, 5)
sns.boxplot(data=df, x='school_tier', y='hotness_index')
plt.title('不同学校层次的热度指数')
plt.xticks(rotation=45)

# 热度指数与科类的关系
plt.subplot(2, 3, 6)
sns.boxplot(data=df, x='category', y='hotness_index')
plt.title('不同科类的热度指数')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# 数值特征分析
numeric_features = ['plan_quota', 'apply_num', 'min_score', 'min_score_rank', 'hotness_index']

plt.figure(figsize=(15, 10))

for i, feature in enumerate(numeric_features[:-1], 1):
    plt.subplot(2, 2, i)
    sns.scatterplot(data=df, x=feature, y='hotness_index', alpha=0.6)
    plt.title(f'{feature} vs 热度指数')
    
    # 添加相关系数
    corr = df[feature].corr(df['hotness_index'])
    plt.text(0.05, 0.95, f'相关系数: {corr:.3f}', 
             transform=plt.gca().transAxes, 
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

# 相关性矩阵
corr_matrix = df[numeric_features].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.3f')
plt.title('数值特征相关性矩阵')
plt.show()

In [ ]:
# 分类特征分析
categorical_features = ['province', 'school_tier', 'category']

for feature in categorical_features:
    plt.figure(figsize=(12, 6))
    
    # 每个类别的平均热度指数
    avg_hotness = df.groupby(feature)['hotness_index'].mean().sort_values(ascending=False)
    
    if feature == 'province':
        # 省份太多，只显示前15和后5
        top_provinces = avg_hotness.head(15)
        bottom_provinces = avg_hotness.tail(5)
        combined = pd.concat([top_provinces, bottom_provinces])
        
        plt.figure(figsize=(14, 8))
        sns.barplot(x=combined.index, y=combined.values)
        plt.title('各省份平均热度指数 (前15 + 后5)')
        plt.xticks(rotation=45)
    else:
        sns.barplot(x=avg_hotness.index, y=avg_hotness.values)
        plt.title(f'{feature} 平均热度指数')
        plt.xticks(rotation=45)
    
    plt.ylabel('平均热度指数')
    plt.tight_layout()
    plt.show()
    
    print(f"\n{feature} 统计:")
    print(f"唯一值数量: {df[feature].nunique()}")
    print(f"各类别数量: {df[feature].value_counts()}")

In [ ]:
# 数值特征的分布分析
numeric_features = ['plan_quota', 'apply_num', 'min_score', 'min_score_rank']

plt.figure(figsize=(15, 10))

for i, feature in enumerate(numeric_features, 1):
    plt.subplot(2, 2, i)
    
    # 原始分布
    sns.histplot(df[feature], bins=30, kde=True, alpha=0.7, label='原始数据')
    
    # 对数变换后的分布
    if feature != 'min_score':  # 分数不需要对数变换
        log_feature = np.log1p(df[feature])
        sns.histplot(log_feature, bins=30, kde=True, alpha=0.5, 
                    label='对数变换', color='orange')
        plt.title(f'{feature} 分布对比')
        plt.legend()
    else:
        plt.title(f'{feature} 分布')
    
    plt.xlabel(feature)

plt.tight_layout()
plt.show()

## 特征工程方案

基于以上分析，确定以下特征工程方案：

In [ ]:
def feature_engineering(df):
    """
    特征工程函数
    """
    df_processed = df.copy()
    
    # 1. 数值特征对数变换
    df_processed['log_plan_quota'] = np.log1p(df_processed['plan_quota'])
    df_processed['log_apply_num'] = np.log1p(df_processed['apply_num'])
    df_processed['log_min_score_rank'] = np.log1p(df_processed['min_score_rank'])
    
    # 2. 分类特征One-hot编码
    categorical_features = ['province', 'school_tier', 'category']
    df_encoded = pd.get_dummies(df_processed, columns=categorical_features, drop_first=True)
    
    # 3. 移除不需要的特征
    features_to_remove = ['school_name', 'major_name', 'plan_quota', 'apply_num', 
                          'min_score_rank', 'hotness_index']
    for feature in features_to_remove:
        if feature in df_encoded.columns:
            df_encoded = df_encoded.drop(feature, axis=1)
    
    return df_processed, df_encoded

# 应用特征工程
df_processed, df_encoded = feature_engineering(df)

print(f"原始数据形状: {df.shape}")
print(f"处理后数据形状: {df_encoded.shape}")
print(f"\n编码后的特征:")
print(df_encoded.columns.tolist())

In [ ]:
# 准备训练数据
X = df_encoded
y = df['hotness_index']

print(f"特征矩阵形状: {X.shape}")
print(f"目标变量形状: {y.shape}")
print(f"\n特征列表:")
for i, col in enumerate(X.columns):
    print(f"{i+1:2d}. {col}")

# 保存特征列表供后续使用
feature_columns = X.columns.tolist()
print(f"\n特征列表长度: {len(feature_columns)}")

In [ ]:
# 特征重要性初步分析
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# 分割数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练简单的随机森林模型来评估特征重要性
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# 特征重要性
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 15 重要特征:")
print(feature_importance.head(15))

# 可视化特征重要性
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
sns.barplot(data=top_features, x='importance', y='feature')
plt.title('Top 15 特征重要性')
plt.xlabel('重要性')
plt.tight_layout()
plt.show()

## EDA总结与特征工程结论

### 主要发现：

1. **目标变量分布**：
   - 热度指数呈右偏分布，大部分集中在2-10之间
   - 对数变换后更接近正态分布

2. **重要特征**：
   - `min_score_rank`（省内排名）与热度指数相关性最高
   - `min_score`（最低分数）也是重要预测因子
   - `school_tier`（学校层次）对热度有显著影响

3. **数据质量**：
   - 数据完整，无缺失值
   - 数值特征分布合理，符合业务逻辑

### 特征工程方案：

1. **数值特征**：
   - 对 `plan_quota`, `apply_num`, `min_score_rank` 进行对数变换
   - `min_score` 保持原值

2. **分类特征**：
   - 对 `province`, `school_tier`, `category` 进行 One-hot 编码
   - 使用 `drop_first=True` 避免多重共线性

3. **移除特征**：
   - 移除标识类特征：`school_name`, `major_name`
   - 移除原始数值特征（已做对数变换）：`plan_quota`, `apply_num`, `min_score_rank`
   - 移除目标变量：`hotness_index`

### 下一步：
特征工程方案已确定，可以开始实现 `train.py` 模型训练脚本。